In [1]:
%cd '/content/drive/MyDrive/UCO/projects/epilepsy'

/content/drive/MyDrive/UCO/projects/epilepsy


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.1 MB/s eta 0:00:00


In [8]:
# prompt: import data_prep.py in the directory
import torch
import sys
sys.path.append('./') # Assuming data_prep.py is in the same directory
print(sys.path)
from model import HeteroGAT3, HeteroGAT4
from data_prep import heterdataPrep
from util import balance_homodata_underSample, load_homodata, train_test_split_homodata
from test_sample_size import test_sample_size

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', './', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/tmp/tmprngnzlwf', './', './', './']


In [19]:
%%time
GAT500, test_data=test_sample_size('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',offset = 500, batch_size = 'full', hidden_channels = 16, convhidden_channels = 16, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([      0,       1,       2,  ..., 1701681, 1701682, 1701683])
tensor([     11,      12,      18,  ..., 1701668, 1701675, 1701677])
tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 

100%|██████████| 800/800 [04:33<00:00,  2.92it/s, loss=0.551]


train performance:
Confusion Matrix:
[[50101 32089]
 [12819 69371]]
Sensitivity (Recall): 0.8440
Specificity: 0.6096
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.61      0.69     82190
           1       0.68      0.84      0.76     82190

    accuracy                           0.73    164380
   macro avg       0.74      0.73      0.72    164380
weighted avg       0.74      0.73      0.72    164380

AUC-ROC: 0.7667
PR-AUC: 0.7233
test performance:
Confusion Matrix:
[[150893 168946]
 [  1769  18728]]
Sensitivity (Recall): 0.9137
Specificity: 0.4718
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.47      0.64    319839
           1       0.10      0.91      0.18     20497

    accuracy                           0.50    340336
   macro avg       0.54      0.69      0.41    340336
weighted avg       0.93      0.50      0.61    340336

AUC-ROC: 0.7191
PR-AUC: 0.0984
C

In [ ]:
test_data

HeteroData(
  diagnosis={ x=[500, 1] },
  person={
    x=[340336, 3],
    y=[340336],
  },
  (person, to, diagnosis)={
    edge_index=[2, 5517432],
    edge_attr=[5517432, 1],
  },
  (diagnosis, to, person)={ edge_index=[2, 5517432] },
  (diagnosis, diag_conn, diagnosis)={ edge_index=[2, 124750] }
)

In [ ]:
GAT500

HeteroGAT4(
  (convs): ModuleList(
    (0-1): 2 x HeteroConv(num_relations=3)
  )
  (gat1): HeteroConv(num_relations=3)
  (gat2): HeteroConv(num_relations=3)
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=32, out_features=32, bias=True)
  (lin3): Linear(in_features=32, out_features=2, bias=True)
  (edge_mlp): Sequential(
    (0): Linear(in_features=1, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (edge_mlp2): Sequential(
    (0): Linear(in_features=1, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
# prompt: save GAT500 to local

import torch

# Assuming GAT500 is a PyTorch model or a tensor
torch.save(GAT500.state_dict(), '/content/drive/MyDrive/UCO/projects/epilepsy/GAT500_model.pth')

# Alternatively, if GAT500 is just a tensor:
# torch.save(GAT500, '/content/drive/MyDrive/UCO/projects/epilepsy/GAT500_tensor.pth')


In [14]:
from data_prep import heterdataPrep, connect_diagnosis_edges
file = '/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5'
offset = 500
data = load_homodata(file)
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

train_data, test_data = train_test_split_homodata(data, 0.8, offset)
train_data = balance_homodata_underSample(train_data, offset)
train_data_homo = train_data.clone()

test_data_balanced = balance_homodata_underSample(test_data, offset)
train_data = heterdataPrep(train_data, offset, agg=False)
test_data = heterdataPrep(test_data, offset, agg=False)
test_data_balanced = heterdataPrep(test_data_balanced, offset, agg=False)

train_data_d2d = connect_diagnosis_edges(train_data)
print(train_data_d2d)
test_data_d2d = connect_diagnosis_edges(test_data)
print(test_data_d2d)
test_data_balanced_d2d = connect_diagnosis_edges(test_data_balanced)

Using device: cuda
tensor([      0,       2,       5,  ..., 1701680, 1701682, 1701683])
tensor([      1,       3,       4,  ..., 1701672, 1701674, 1701681])
tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 

In [15]:
# Load the saved model's state dictionary
GAT500_state_dict = torch.load('/content/drive/MyDrive/UCO/projects/epilepsy/GAT500_model.pth')

# Assuming you have the model architecture defined (HeteroGAT3)
model = HeteroGAT4(input_features=3, edge_features=1, hidden_channels=32,
                          convhidden_channels=32, out_channels=2, num_heads=4,
                          metadata=train_data_d2d.metadata(), num_layers=1)

# Load the state dictionary into the model
model.load_state_dict(GAT500_state_dict)

# Now the model 'model' contains the loaded GAT500


RuntimeError: Error(s) in loading state_dict for HeteroGAT4:
	Unexpected key(s) in state_dict: "convs.1.convs.<person___to___diagnosis>.att_src", "convs.1.convs.<person___to___diagnosis>.att_dst", "convs.1.convs.<person___to___diagnosis>.bias", "convs.1.convs.<person___to___diagnosis>.lin_src.weight", "convs.1.convs.<person___to___diagnosis>.lin_dst.weight", "convs.1.convs.<diagnosis___to___person>.att_src", "convs.1.convs.<diagnosis___to___person>.att_dst", "convs.1.convs.<diagnosis___to___person>.bias", "convs.1.convs.<diagnosis___to___person>.lin_src.weight", "convs.1.convs.<diagnosis___to___person>.lin_dst.weight", "convs.1.convs.<diagnosis___diag_conn___diagnosis>.att_src", "convs.1.convs.<diagnosis___diag_conn___diagnosis>.att_dst", "convs.1.convs.<diagnosis___diag_conn___diagnosis>.bias", "convs.1.convs.<diagnosis___diag_conn___diagnosis>.lin_src.weight", "convs.1.convs.<diagnosis___diag_conn___diagnosis>.lin_dst.weight". 
	size mismatch for gat1.convs.<person___to___diagnosis>.

In [ ]:
# prompt: label which patient had accurate prediction by the model

# Assuming 'test_data' contains the ground truth labels and model predictions
# and it's a dictionary or a similar data structure.

# Example: Assuming test_data has 'y_true' and 'y_pred' keys
# for ground truth and predicted labels respectively.

accurate_predictions = []

for i in range(len(test_data['y_true'])):
  if test_data['y_true'][i] == test_data['y_pred'][i]:
    accurate_predictions.append(i)

print("Patients with accurate predictions:", accurate_predictions)


Patients with accurate predictions: []


In [ ]:
from torch_geometric.explain import Explainer, GNNExplainer
data = test_data
model = GAT20
explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)
node_index = 10
# Get the edge_attr_dict from your data
edge_attr_dict = data.edge_attr_dict  # Assuming your data has edge attributes

# Pass edge_attr_dict to the explainer along with other arguments
explanation = explainer(data['diagnosis'].x, data['person','to','diagnosis'].edge_index,
                        edge_attr_dict=edge_attr_dict,  # Pass edge_attr_dict here
                        index=node_index)
print(f'Generated explanations in {explanation.available_explanations}')

path = 'feature_importance.png'
explanation.visualize_feature_importance(path, top_k=10)
print(f"Feature importance plot has been saved to '{path}'")

RuntimeError: Tensor.__contains__ only supports Tensor or scalar, but you passed in a <class 'tuple'>.

In [ ]:
GAT20=test_sample_size('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag20_first.h5',offset = 20, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

KeyboardInterrupt: 

In [ ]:
GAT20=test_sample_size('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag20_first.h5',offset = 20, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      0,       1,       2,  ..., 1701681, 1701682, 1701683])
tensor([      4,       5,       9,  ..., 1701671, 1701672, 1701676])
class counts =  tensor([1278929,   82419])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([320068,  20268])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [02:28<00:00,  5.40it/s, loss=0.636]


train performance:
Confusion Matrix:
[[28897 53522]
 [ 8651 73768]]
Sensitivity (Recall): 0.8950
Specificity: 0.3506
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.35      0.48     82419
           1       0.58      0.90      0.70     82419

    accuracy                           0.62    164838
   macro avg       0.67      0.62      0.59    164838
weighted avg       0.67      0.62      0.59    164838

AUC-ROC: 0.6499
PR-AUC: 0.6398
test performance:
Confusion Matrix:
[[127131 192937]
 [  3107  17161]]
Sensitivity (Recall): 0.8467
Specificity: 0.3972
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.40      0.56    320068
           1       0.08      0.85      0.15     20268

    accuracy                           0.42    340336
   macro avg       0.53      0.62      0.36    340336
weighted avg       0.92      0.42      0.54    340336

AUC-ROC: 0.6477
PR-AUC: 0.1036


In [ ]:
GAT50=test_sample_size('EPI_heterodata_diag50_first.h5',offset = 50, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([      0,       1,       2,  ..., 1701680, 1701681, 1701682])
tensor([     12,      15,      36,  ..., 1701676, 1701679, 1701683])
class counts =  tensor([1279414,   81934])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319583,  20753])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [03:33<00:00,  3.75it/s, loss=0.592]


train performance:
Confusion Matrix:
[[41854 40080]
 [12107 69827]]
Sensitivity (Recall): 0.8522
Specificity: 0.5108
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.51      0.62     81934
           1       0.64      0.85      0.73     81934

    accuracy                           0.68    163868
   macro avg       0.71      0.68      0.67    163868
weighted avg       0.71      0.68      0.67    163868

AUC-ROC: 0.7168
PR-AUC: 0.6971
test performance:
Confusion Matrix:
[[178686 140897]
 [  4730  16023]]
Sensitivity (Recall): 0.7721
Specificity: 0.5591
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.56      0.71    319583
           1       0.10      0.77      0.18     20753

    accuracy                           0.57    340336
   macro avg       0.54      0.67      0.45    340336
weighted avg       0.92      0.57      0.68    340336

AUC-ROC: 0.7044
PR-AUC: 0.1387


In [ ]:
GAT50=test_sample_size('EPI_heterodata_diag50_first.h5',offset = 50, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      1,       2,       3,  ..., 1701681, 1701682, 1701683])
tensor([      0,      17,      21,  ..., 1701663, 1701671, 1701676])
class counts =  tensor([1279070,   82278])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319927,  20409])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [03:33<00:00,  3.75it/s, loss=0.611]


train performance:
Confusion Matrix:
[[38955 43323]
 [11794 70484]]
Sensitivity (Recall): 0.8567
Specificity: 0.4735
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.47      0.59     82278
           1       0.62      0.86      0.72     82278

    accuracy                           0.67    164556
   macro avg       0.69      0.67      0.65    164556
weighted avg       0.69      0.67      0.65    164556

AUC-ROC: 0.6888
PR-AUC: 0.6339
test performance:
Confusion Matrix:
[[169447 150480]
 [  4233  16176]]
Sensitivity (Recall): 0.7926
Specificity: 0.5296
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.53      0.69    319927
           1       0.10      0.79      0.17     20409

    accuracy                           0.55    340336
   macro avg       0.54      0.66      0.43    340336
weighted avg       0.92      0.55      0.66    340336

AUC-ROC: 0.6924
PR-AUC: 0.1127


In [ ]:
GAT100 = test_sample_size('EPI_heterodata_diag100_first.h5',offset = 100, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
class counts =  tensor([1279083,   82265])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319915,  20422])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [04:42<00:00,  2.83it/s, loss=0.571]


train performance:
Confusion Matrix:
[[45924 36341]
 [12410 69855]]
Sensitivity (Recall): 0.8491
Specificity: 0.5582
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.56      0.65     82265
           1       0.66      0.85      0.74     82265

    accuracy                           0.70    164530
   macro avg       0.72      0.70      0.70    164530
weighted avg       0.72      0.70      0.70    164530

AUC-ROC: 0.7423
PR-AUC: 0.7188
test performance:
Confusion Matrix:
[[200773 119142]
 [  5973  14449]]
Sensitivity (Recall): 0.7075
Specificity: 0.6276
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.63      0.76    319915
           1       0.11      0.71      0.19     20422

    accuracy                           0.63    340337
   macro avg       0.54      0.67      0.48    340337
weighted avg       0.92      0.63      0.73    340337

AUC-ROC: 0.7078
PR-AUC: 0.1406


In [ ]:
GAT100 = test_sample_size('EPI_heterodata_diag100_first.h5',offset = 100, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      0,       1,       2,  ..., 1701681, 1701682, 1701683])
tensor([     10,      11,      18,  ..., 1701657, 1701659, 1701673])
class counts =  tensor([1279266,   82082])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319731,  20605])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [04:35<00:00,  2.91it/s, loss=0.584]


train performance:
Confusion Matrix:
[[43723 38359]
 [12328 69754]]
Sensitivity (Recall): 0.8498
Specificity: 0.5327
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.53      0.63     82082
           1       0.65      0.85      0.73     82082

    accuracy                           0.69    164164
   macro avg       0.71      0.69      0.68    164164
weighted avg       0.71      0.69      0.68    164164

AUC-ROC: 0.7293
PR-AUC: 0.6993
test performance:
Confusion Matrix:
[[176609 143122]
 [  3670  16935]]
Sensitivity (Recall): 0.8219
Specificity: 0.5524
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.55      0.71    319731
           1       0.11      0.82      0.19     20605

    accuracy                           0.57    340336
   macro avg       0.54      0.69      0.45    340336
weighted avg       0.93      0.57      0.68    340336

AUC-ROC: 0.7237
PR-AUC: 0.1371


In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
class counts =  tensor([1279151,   82197])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319847,  20490])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [06:13<00:00,  2.14it/s, loss=0.509]


train performance:
Confusion Matrix:
[[51682 30515]
 [10486 71711]]
Sensitivity (Recall): 0.8724
Specificity: 0.6288
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.63      0.72     82197
           1       0.70      0.87      0.78     82197

    accuracy                           0.75    164394
   macro avg       0.77      0.75      0.75    164394
weighted avg       0.77      0.75      0.75    164394

AUC-ROC: 0.8056
PR-AUC: 0.7987
test performance:
Confusion Matrix:
[[203828 116019]
 [  3157  17333]]
Sensitivity (Recall): 0.8459
Specificity: 0.6373
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.64      0.77    319847
           1       0.13      0.85      0.23     20490

    accuracy                           0.65    340337
   macro avg       0.56      0.74      0.50    340337
weighted avg       0.93      0.65      0.74    340337

AUC-ROC: 0.7903
PR-AUC: 0.2181


In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
class counts =  tensor([1279144,   82204])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319854,  20483])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [06:12<00:00,  2.15it/s, loss=0.512]


train performance:
Confusion Matrix:
[[51974 30230]
 [11105 71099]]
Sensitivity (Recall): 0.8649
Specificity: 0.6323
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.63      0.72     82204
           1       0.70      0.86      0.77     82204

    accuracy                           0.75    164408
   macro avg       0.76      0.75      0.75    164408
weighted avg       0.76      0.75      0.75    164408

AUC-ROC: 0.8043
PR-AUC: 0.7995
test performance:
Confusion Matrix:
[[216249 103605]
 [  6669  13814]]
Sensitivity (Recall): 0.6744
Specificity: 0.6761
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.68      0.80    319854
           1       0.12      0.67      0.20     20483

    accuracy                           0.68    340337
   macro avg       0.54      0.68      0.50    340337
weighted avg       0.92      0.68      0.76    340337

AUC-ROC: 0.7302
PR-AUC: 0.1535


In [ ]:
GAT500 = test_sample_size('EPI_heterodata_diag500_first.h5',offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
class counts =  tensor([1279228,   82120])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319770,  20567])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [08:06<00:00,  1.64it/s, loss=0.521]


train performance:
Confusion Matrix:
[[54525 27595]
 [12163 69957]]
Sensitivity (Recall): 0.8519
Specificity: 0.6640
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.66      0.73     82120
           1       0.72      0.85      0.78     82120

    accuracy                           0.76    164240
   macro avg       0.77      0.76      0.76    164240
weighted avg       0.77      0.76      0.76    164240

AUC-ROC: 0.8049
PR-AUC: 0.7939
test performance:
Confusion Matrix:
[[222124  97646]
 [  3984  16583]]
Sensitivity (Recall): 0.8063
Specificity: 0.6946
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.69      0.81    319770
           1       0.15      0.81      0.25     20567

    accuracy                           0.70    340337
   macro avg       0.56      0.75      0.53    340337
weighted avg       0.93      0.70      0.78    340337

AUC-ROC: 0.8021
PR-AUC: 0.2578


In [ ]:
GAT500 = test_sample_size('EPI_heterodata_diag500_first.h5',offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      0,       2,       3,  ..., 1701680, 1701681, 1701682])
tensor([      1,       4,      15,  ..., 1701667, 1701679, 1701683])
class counts =  tensor([1279150,   82198])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319847,  20489])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [07:59<00:00,  1.67it/s, loss=0.551]


train performance:
Confusion Matrix:
[[52265 29933]
 [13918 68280]]
Sensitivity (Recall): 0.8307
Specificity: 0.6358
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.64      0.70     82198
           1       0.70      0.83      0.76     82198

    accuracy                           0.73    164396
   macro avg       0.74      0.73      0.73    164396
weighted avg       0.74      0.73      0.73    164396

AUC-ROC: 0.7740
PR-AUC: 0.7347
test performance:
Confusion Matrix:
[[223510  96337]
 [  5803  14686]]
Sensitivity (Recall): 0.7168
Specificity: 0.6988
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.70      0.81    319847
           1       0.13      0.72      0.22     20489

    accuracy                           0.70    340336
   macro avg       0.55      0.71      0.52    340336
weighted avg       0.92      0.70      0.78    340336

AUC-ROC: 0.7609
PR-AUC: 0.1754


In [ ]:
GAT500 = test_sample_size('EPI_heterodata_diag500_first.h5',offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      0,       1,       2,  ..., 1701681, 1701682, 1701683])
tensor([      7,       8,       9,  ..., 1701670, 1701671, 1701676])
class counts =  tensor([1279074,   82274])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319923,  20413])
minority_class:  tensor(1)
majority_class:  tensor(0)
HeteroData(
  diagnosis={ x=[500, 1] },
  person={
    x=[164548, 3],
    y=[164548],
  },
  (person, to, diagnosis)={
    edge_index=[2, 2914510],
    edge_attr=[2914510, 1],
  },
  (diagnosis, to, person)={ edge_index=[2, 2914510] },
  (diagnosis, diag_conn, diagnosis)={ edge_index=[2, 124750] }
)
HeteroData(
  diagnosis={ x=[500, 1] },
  person={
    x=[340336, 3],
    y=[340336],
  },
  (person, to, diagnosis)={
    edge_index=[2, 5498532],
    edge_attr=[5498532, 1],
  },
  (diagnosis, to, person)={ edge_index=[2, 5498532] },
  (diagnosis, diag_conn, diagnosis)={ edge_index=[2, 124750] }
)


  0%|          | 0/800 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 1, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([      3,       4,       5,  ..., 1701680, 1701681, 1701682])
tensor([      0,       1,       2,  ..., 1701671, 1701673, 1701683])
class counts =  tensor([1279267,   82081])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319730,  20606])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [02:14<00:00,  5.94it/s, loss=0.6]


train performance:
Confusion Matrix:
[[48092 33989]
 [17243 64838]]
Sensitivity (Recall): 0.7899
Specificity: 0.5859
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.59      0.65     82081
           1       0.66      0.79      0.72     82081

    accuracy                           0.69    164162
   macro avg       0.70      0.69      0.68    164162
weighted avg       0.70      0.69      0.68    164162

AUC-ROC: 0.7211
PR-AUC: 0.6696
test performance:
Confusion Matrix:
[[190490 129240]
 [  4394  16212]]
Sensitivity (Recall): 0.7868
Specificity: 0.5958
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.60      0.74    319730
           1       0.11      0.79      0.20     20606

    accuracy                           0.61    340336
   macro avg       0.54      0.69      0.47    340336
weighted avg       0.93      0.61      0.71    340336

AUC-ROC: 0.7212
PR-AUC: 0.1168


In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 2, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([      0,       3,       4,  ..., 1701681, 1701682, 1701683])
tensor([      1,       2,      10,  ..., 1701662, 1701664, 1701680])
class counts =  tensor([1279410,   81938])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319587,  20749])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [03:34<00:00,  3.73it/s, loss=0.522]


train performance:
Confusion Matrix:
[[51020 30918]
 [11023 70915]]
Sensitivity (Recall): 0.8655
Specificity: 0.6227
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.62      0.71     81938
           1       0.70      0.87      0.77     81938

    accuracy                           0.74    163876
   macro avg       0.76      0.74      0.74    163876
weighted avg       0.76      0.74      0.74    163876

AUC-ROC: 0.7970
PR-AUC: 0.7900
test performance:
Confusion Matrix:
[[209807 109780]
 [  5427  15322]]
Sensitivity (Recall): 0.7384
Specificity: 0.6565
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.66      0.78    319587
           1       0.12      0.74      0.21     20749

    accuracy                           0.66    340336
   macro avg       0.55      0.70      0.50    340336
weighted avg       0.92      0.66      0.75    340336

AUC-ROC: 0.7474
PR-AUC: 0.1248


In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 6, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([      0,       1,       2,  ..., 1701678, 1701682, 1701683])
tensor([      7,       8,      17,  ..., 1701679, 1701680, 1701681])
class counts =  tensor([1279235,   82113])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319762,  20574])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [08:59<00:00,  1.48it/s, loss=0.504]


train performance:
Confusion Matrix:
[[52150 29963]
 [10514 71599]]
Sensitivity (Recall): 0.8720
Specificity: 0.6351
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.64      0.72     82113
           1       0.70      0.87      0.78     82113

    accuracy                           0.75    164226
   macro avg       0.77      0.75      0.75    164226
weighted avg       0.77      0.75      0.75    164226

AUC-ROC: 0.8088
PR-AUC: 0.8077
test performance:
Confusion Matrix:
[[204313 115449]
 [  3265  17309]]
Sensitivity (Recall): 0.8413
Specificity: 0.6390
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.64      0.77    319762
           1       0.13      0.84      0.23     20574

    accuracy                           0.65    340336
   macro avg       0.56      0.74      0.50    340336
weighted avg       0.93      0.65      0.74    340336

AUC-ROC: 0.7879
PR-AUC: 0.2282


In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 1, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      0,       1,       2,  ..., 1701680, 1701681, 1701682])
tensor([      4,      14,      17,  ..., 1701667, 1701678, 1701683])
class counts =  tensor([1279175,   82173])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319822,  20514])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [02:17<00:00,  5.83it/s, loss=0.634]


train performance:
Confusion Matrix:
[[52803 29370]
 [27577 54596]]
Sensitivity (Recall): 0.6644
Specificity: 0.6426
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.64      0.65     82173
           1       0.65      0.66      0.66     82173

    accuracy                           0.65    164346
   macro avg       0.65      0.65      0.65    164346
weighted avg       0.65      0.65      0.65    164346

AUC-ROC: 0.6849
PR-AUC: 0.7387
test performance:
Confusion Matrix:
[[205633 114189]
 [  6905  13609]]
Sensitivity (Recall): 0.6634
Specificity: 0.6430
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.64      0.77    319822
           1       0.11      0.66      0.18     20514

    accuracy                           0.64    340336
   macro avg       0.54      0.65      0.48    340336
weighted avg       0.92      0.64      0.74    340336

AUC-ROC: 0.6822
PR-AUC: 0.3647


In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 2, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      0,       2,       3,  ..., 1701681, 1701682, 1701683])
tensor([      1,       8,      12,  ..., 1701663, 1701672, 1701680])
class counts =  tensor([1279230,   82118])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319767,  20569])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [03:32<00:00,  3.76it/s, loss=0.579]


train performance:
Confusion Matrix:
[[45982 36136]
 [13251 68867]]
Sensitivity (Recall): 0.8386
Specificity: 0.5600
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.56      0.65     82118
           1       0.66      0.84      0.74     82118

    accuracy                           0.70    164236
   macro avg       0.72      0.70      0.69    164236
weighted avg       0.72      0.70      0.69    164236

AUC-ROC: 0.7340
PR-AUC: 0.6730
test performance:
Confusion Matrix:
[[208394 111373]
 [  5703  14866]]
Sensitivity (Recall): 0.7227
Specificity: 0.6517
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.65      0.78    319767
           1       0.12      0.72      0.20     20569

    accuracy                           0.66    340336
   macro avg       0.55      0.69      0.49    340336
weighted avg       0.92      0.66      0.75    340336

AUC-ROC: 0.7367
PR-AUC: 0.1317


In [ ]:
GAT200 = test_sample_size('EPI_heterodata_diag200_first.h5',offset = 200, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 6, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = False)

Using device: cuda
tensor([      0,       1,       3,  ..., 1701679, 1701680, 1701683])
tensor([      2,      15,      17,  ..., 1701672, 1701681, 1701682])
class counts =  tensor([1279186,   82162])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319811,  20525])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [08:59<00:00,  1.48it/s, loss=0.507]


train performance:
Confusion Matrix:
[[51382 30780]
 [10007 72155]]
Sensitivity (Recall): 0.8782
Specificity: 0.6254
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.63      0.72     82162
           1       0.70      0.88      0.78     82162

    accuracy                           0.75    164324
   macro avg       0.77      0.75      0.75    164324
weighted avg       0.77      0.75      0.75    164324

AUC-ROC: 0.8057
PR-AUC: 0.7984
test performance:
Confusion Matrix:
[[217895 101916]
 [  5692  14833]]
Sensitivity (Recall): 0.7227
Specificity: 0.6813
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.68      0.80    319811
           1       0.13      0.72      0.22     20525

    accuracy                           0.68    340336
   macro avg       0.55      0.70      0.51    340336
weighted avg       0.92      0.68      0.77    340336

AUC-ROC: 0.7555
PR-AUC: 0.1666
